# 1. Overview & Objectives  
Brief description of what this notebook is responsible for.

- What type of models are implemented here
    - LSTM
    - NBEATS
    - NHITS

- Expected outputs (e.g., CSV with metrics, plots, forecasts)

All models should be implemented in such a way that in the visualizations.ipynb it will be possible to call a function for the model using the parameters from the `best_param.csv` file for easy of plotting on the time series. Should be possible for test & val

All the best models should be saved in the `best_param.csv` file


# 2. Imports & Setup

In [39]:
# Importing the helper notebooks

## Enable imports from .ipynb files
import import_ipynb
import sys
sys.path.append("code")

## Importing the helper notebooks as modules
from splitting import split_time_series
from metrics import evaluate_and_save, load_best_models

# Notebook specific imports
# Core libraries
import numpy as np
import pandas as pd
from pathlib import Path

# Nixtla
from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM, NBEATS, NHITS, TFT
from neuralforecast.losses.pytorch import MAE

# Torch (for reproducibility)
import torch
torch.manual_seed(42)

# 3. Load Data & Train/Val/Test Split
use `split_time_series()`

In [40]:
splits = split_time_series()

train_df = splits["train"]
val_df   = splits["val"]
test_df  = splits["test"]

TARGET_COL = "tavg"

In [41]:
def to_nixtla_format(df, uid="series_1"):
    out = df.rename(columns={"time": "ds", "tavg": "y"}).copy()
    out["unique_id"] = uid
    return out[["unique_id", "ds", "y"]]

train_nf = to_nixtla_format(train_df)
val_nf   = to_nixtla_format(val_df)
test_nf  = to_nixtla_format(test_df)

# 4. Model Definition  
Clearly specify:  
- Model names  
- Hyperparameters  

In [42]:
HORIZON = len(val_nf)
INPUT_SIZE = 365
MAX_STEPS = 500

def make_name(base, **params):
    return "_".join([base] + [f"{k}{v}" for k, v in params.items()])

In [43]:
models = [
    (
        make_name("LSTM", h=HORIZON, in_=INPUT_SIZE, enc="64x2"),
        LSTM(
            h=HORIZON,
            input_size=INPUT_SIZE,
            encoder_hidden_size=64,
            encoder_n_layers=2,
            decoder_hidden_size=64,
            decoder_layers=2,
            loss=MAE(),
            max_steps=MAX_STEPS
        )
    ),

    (
        make_name("NBEATS", h=HORIZON, in_=INPUT_SIZE),
        NBEATS(
            h=HORIZON,
            input_size=INPUT_SIZE,
            loss=MAE(),
            max_steps=MAX_STEPS
        )
    ),

    (
        make_name("NHITS", h=HORIZON, in_=INPUT_SIZE),
        NHITS(
            h=HORIZON,
            input_size=INPUT_SIZE,
            loss=MAE(),
            max_steps=MAX_STEPS
        )
    )
]


Seed set to 1
Seed set to 1
Seed set to 1


# 5. Training  
For each model:  
- Fit on training data  
- (For ML & Neural) prepare features / loaders / windows

## 5.1 For validation

In [44]:
H_VAL = len(val_nf)

models_val = [
    (
        make_name("LSTM", h=H_VAL, in_=INPUT_SIZE, enc="64x2"),
        LSTM(
            h=H_VAL,
            input_size=INPUT_SIZE,
            encoder_hidden_size=64,
            encoder_n_layers=2,
            decoder_hidden_size=64,
            decoder_layers=2,
            loss=MAE(),
            max_steps=MAX_STEPS
        )
    ),

    (
        make_name("NBEATS", h=H_VAL, in_=INPUT_SIZE),
        NBEATS(
            h=H_VAL,
            input_size=INPUT_SIZE,
            loss=MAE(),
            max_steps=MAX_STEPS
        )
    ),

    (
        make_name("NHITS", h=H_VAL, in_=INPUT_SIZE),
        NHITS(
            h=H_VAL,
            input_size=INPUT_SIZE,
            loss=MAE(),
            max_steps=MAX_STEPS
        )
    )
]


nf_val = NeuralForecast(
    models=[m for _, m in models_val],
    freq="D"
)

Seed set to 1
Seed set to 1
Seed set to 1


## 5.2 For Test

In [45]:
H_TEST = len(test_nf)

models_test = [
    (
        make_name("LSTM", h=H_TEST, in_=INPUT_SIZE, enc="64x2"),
        LSTM(
            h=H_TEST,
            input_size=INPUT_SIZE,
            encoder_hidden_size=64,
            encoder_n_layers=2,
            decoder_hidden_size=64,
            decoder_layers=2,
            loss=MAE(),
            max_steps=MAX_STEPS
        )
    ),

    (
        make_name("NBEATS", h=H_TEST, in_=INPUT_SIZE),
        NBEATS(
            h=H_TEST,
            input_size=INPUT_SIZE,
            loss=MAE(),
            max_steps=MAX_STEPS
        )
    ),

    (
        make_name("NHITS", h=H_TEST, in_=INPUT_SIZE),
        NHITS(
            h=H_TEST,
            input_size=INPUT_SIZE,
            loss=MAE(),
            max_steps=MAX_STEPS
        )
    )
]


nf_test = NeuralForecast(
    models=[m for _, m in models_test],
    freq="D"
)

Seed set to 1
Seed set to 1
Seed set to 1


# 6. Forecasting  
- Produce forecasts for validation and test horizons

## 6.1 Validation Forecast

In [46]:
nf_val.fit(train_nf)
val_forecasts = nf_val.predict()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name              | Type          | Params | Mode 
------------------------------------------------------------
0 | loss              | MAE           | 0      | train
1 | padder_train      | ConstantPad1d | 0      | train
2 | scaler            | TemporalNorm  | 0      | train
3 | hist_encoder      | LSTM          | 50.4 K | train
4 | mlp_decoder       | MLP           | 4.2 K  | train
5 | upsample_sequence | Linear        | 668 K  | train
------------------------------------------------------------
723 K     Trainable params
0         Non-trainable params
723 K     Total params
2.893     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


Predicting: |          | 0/? [00:00<?, ?it/s]

In [52]:
val_forecasts = val_forecasts.rename(
    columns={
        old: name
        for (name, _), old in zip(models_val, val_forecasts.columns[2:])
    }
)


## 6.2 Test Forecast

In [47]:
nf_test.fit(train_val_nf)
test_forecasts = nf_test.predict()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name              | Type          | Params | Mode 
------------------------------------------------------------
0 | loss              | MAE           | 0      | train
1 | padder_train      | ConstantPad1d | 0      | train
2 | scaler            | TemporalNorm  | 0      | train
3 | hist_encoder      | LSTM          | 50.4 K | train
4 | mlp_decoder       | MLP           | 4.2 K  | train
5 | upsample_sequence | Linear        | 655 K  | train
------------------------------------------------------------
710 K     Trainable params
0         Non-trainable params
710 K     Total params
2.842     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


Predicting: |          | 0/? [00:00<?, ?it/s]

In [51]:
test_forecasts = test_forecasts.rename(
    columns={
        old: name
        for (name, _), old in zip(models_test, test_forecasts.columns[2:])
    }
)

# 7. Evaluation (Using Shared Metrics Function)  
- Apply `evaluate_and_save()` to each model  
- Save results as CSV into `data/models/`  
- Display sorted results table  

In [53]:
OUT_FILE = "neural_models_results.csv"
BEST_PARAM_FILE = "best_param.csv"

results = []
best_params = []

def evaluate_split(y_true, forecasts, split_name):
    for model_name in forecasts.columns:
        if model_name in ["unique_id", "ds"]:
            continue

        metrics = evaluate_and_save(
            y_true=y_true,
            y_pred=forecasts[model_name].values,
            model_name=model_name,
            impl_name="neural",
            split_name=split_name,
            out_filename=OUT_FILE
        )

        results.append(metrics)

        best_params.append({
            "Model": model_name,
            "Impl": "neural"
        })

In [57]:
# Validation
evaluate_split(
    y_true=val_nf["y"].values,
    forecasts=val_forecasts,
    split_name="val"
)

In [58]:
# Test
evaluate_split(
    y_true=test_nf["y"].values,
    forecasts=test_forecasts,
    split_name="test"
)

In [62]:
load_best_models("neural_models_results.csv", split="test")

,Model,Impl,Split,MAE,RMSE,MAPE,OPE,R2
0,LSTM_h1792_in_365_enc64x2,neural,test,3.514066,4.464777,7.235003e+09,0.117615,0.810603
1,NBEATS_h1792_in_365,neural,test,3.552858,4.539296,7.437694e+09,0.159560,0.804228
2,NHITS_h1792_in_365,neural,test,3.572262,4.573224,6.763618e+09,0.129208,0.801291


# 8. Conclusions  
Short wrap-up:  
- Which model family performed best here?  
- Any issues or instability?  
- Notes for integration in the final report  

Best performing model was **LSTM**

With parameters:
```
h=HORIZON,
input_size=INPUT_SIZE,
encoder_hidden_size=64,
encoder_n_layers=2,
decoder_hidden_size=64,
decoder_layers=2,
loss=MAE(),
max_steps=MAX_STEPS
```

MAE: 
of 3.5 degrees